In [2]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pandas as pd
from sovereign.flood import simple_risk_overlay, flopros_risk_overlay

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set filepaths and provide data info

In [3]:
# Set filepaths
root = Path.cwd().parent # find project root
flood_dir = os.path.join(root, 'inputs', 'flood', 'maps')
exposure_dir = os.path.join(root, 'outputs', 'exposure')
vulnerability_path = os.path.join(root, 'inputs', 'flood', 'vulnerability', 'jrc_depth_damage.csv')
# Dictionary of flood maps
flood_dic = {10: 'THA_jrc-flood_RP10.tif',
            20: 'THA_jrc-flood_RP20.tif',
            50: 'THA_jrc-flood_RP50.tif',
            75: 'THA_jrc-flood_RP75.tif',
            100: 'THA_jrc-flood_RP100.tif',
            200: 'THA_jrc-flood_RP200.tif',
            500: 'THA_jrc-flood_RP500.tif'}